In [1]:
import re
import torch
import torch.nn.functional as F
from lib.utils import load, preprocess, get_dict, ask_dict

In [2]:
NGRAM_LENGTH_MIN = 1 # uni-gram
NGRAM_CONTEXT_WINDOW = 2

END_TOKEN = "</s>"
WITH_END_TOKEN = True

## Preprocessing

In [17]:
list_str = load('./data/songs.json')
del list_str["rows"][49] # Shape of You
del list_str["rows"][50 - 1] # You Need Me, I Don't Need You
del list_str["rows"][70 - 2] # Dive
del list_str["rows"][76 - 3] # Take Me Back to London

list_str = [preprocess(str["row"]["text"]) for str in list_str["rows"] if str["row"]["text"]]

if WITH_END_TOKEN:
    # Treat new lines as a word or not
    list_str = [str.replace("\n", f" {END_TOKEN} ") for str in list_str]

{'features': [{'feature_idx': 0, 'name': 'text', 'type': {'dtype': 'string', '_type': 'Value'}}], 'rows': [{'row_idx': 0, 'row': {'text': 'Shape of You Lyrics\nThe club isnt the best place to find a lover\nSo the bar is where I go\nMe and my friends at the table doing shots\nDrinking fast and then we talk slow\nAnd you come over and start up a conversation with just me\nAnd trust me Ill give it a chance now\nTake my hand, stop, put Van the Man on the jukebox\nAnd then we start to dance, and now Im singing like\nGirl, you know I want your love\nYour love was handmade for somebody like me\nCome on now, follow my lead\nI may be crazy, dont mind me\nSay, boy, lets not talk too much\nGrab on my waist and put that body on me\nCome on now, follow my lead\nCome, come on now, follow my lead\nIm in love with the shape of you\nWe push and pull like a magnet do\nAlthough my heart is falling too\nIm in love with your body\nAnd last night you were in my room\nAnd now my bed sheets smell like you\nEv

## Intializations

In [4]:
# Split the lines into words and lowercase them
list_list_words = [re.split(r'[\s\n]+', str.lower().strip()) for str in list_str]
list_str_comp = ["+".join(list_words) for list_words in list_list_words]

print(list_list_words[0])

['the', 'club', 'isnt', 'the', 'best', 'place', 'to', 'find', 'a', 'lover', '</s>', 'so', 'the', 'bar', 'is', 'where', 'i', 'go', '</s>', 'me', 'and', 'my', 'friends', 'at', 'the', 'table', 'doing', 'shots', '</s>', 'drinking', 'fast', 'and', 'then', 'we', 'talk', 'slow', '</s>', 'and', 'you', 'come', 'over', 'and', 'start', 'up', 'a', 'conversation', 'with', 'just', 'me', '</s>', 'and', 'trust', 'me', 'ill', 'give', 'it', 'a', 'chance', 'now', '</s>', 'take', 'my', 'hand', 'stop', 'put', 'van', 'the', 'man', 'on', 'the', 'jukebox', '</s>', 'and', 'then', 'we', 'start', 'to', 'dance', 'and', 'now', 'im', 'singing', 'like', '</s>', 'girl', 'you', 'know', 'i', 'want', 'your', 'love', '</s>', 'your', 'love', 'was', 'handmade', 'for', 'somebody', 'like', 'me', '</s>', 'come', 'on', 'now', 'follow', 'my', 'lead', '</s>', 'i', 'may', 'be', 'crazy', 'dont', 'mind', 'me', '</s>', 'say', 'boy', 'lets', 'not', 'talk', 'too', 'much', '</s>', 'grab', 'on', 'my', 'waist', 'and', 'put', 'that', 'bod

In [5]:
lengths = [len(list_words) for list_words in list_list_words]
lengths_sorted = sorted(lengths)

# The biggest n-gram can only be the length of the smallest song
NGRAM_LENGTH_MAX = lengths_sorted[0]

print(min(lengths), max(lengths))
print(NGRAM_LENGTH_MIN, NGRAM_LENGTH_MAX)

182 1904
1 182


## Dictionary

In [14]:
dict_key_tuple = get_dict(list_list_words, 3)
list_key_tuple_answers = ask_dict(dict_key_tuple, "i+dont")
dicts = {}

list_key_tuple_answers

[('i+dont+need', 69, 0.0013790898007315171, 0.34328358208955223),
 ('i+dont+wanna', 51, 0.0010193272440189474, 0.2537313432835821),
 ('i+dont+want', 23, 0.00045969660024383906, 0.11442786069651742),
 ('i+dont+care', 9, 0.00017988127835628486, 0.04477611940298507),
 ('i+dont+know', 8, 0.00015989446965003097, 0.03980099502487562),
 ('i+dont+love', 7, 0.0001399076609437771, 0.03482587064676617),
 ('i+dont+deserve', 4, 7.994723482501548e-05, 0.01990049751243781),
 ('i+dont+like', 4, 7.994723482501548e-05, 0.01990049751243781),
 ('i+dont+even', 4, 7.994723482501548e-05, 0.01990049751243781),
 ('i+dont+really', 4, 7.994723482501548e-05, 0.01990049751243781),
 ('i+dont+ever', 3, 5.9960426118761616e-05, 0.014925373134328358),
 ('i+dont+but', 2, 3.997361741250774e-05, 0.009950248756218905),
 ('i+dont+mix', 2, 3.997361741250774e-05, 0.009950248756218905),
 ('i+dont+do', 2, 3.997361741250774e-05, 0.009950248756218905),
 ('i+dont+get', 2, 3.997361741250774e-05, 0.009950248756218905),
 ('i+dont+rec

In [11]:
def predict(list_list_words: list, input: str, seed = None, max_lines = 10, separator = "+"):
    if len(input) == 0:
        return "Input cannot be empty"
    list_inputs = input.split(separator)
    ngram_context_window = len(list_inputs) + 1
    
    # Provided that the input is sanitized already
    arg = input
    max_index = 0
    generator = torch.Generator().manual_seed(seed) if seed else None

    # Generate the dictionary
    dict_key_tuple = dicts.get(ngram_context_window) or get_dict(list_list_words, ngram_context_window)
    if ngram_context_window not in dicts:
        dicts[ngram_context_window] = dict_key_tuple

    # Print over all list_words as a starter
    print("START\n")
    for _input in list_inputs:
        print(_input, end=" ")

    while True:
        # Get normalized probabilities for that input
        list_key_tuple_answers = ask_dict(dict_key_tuple, arg)
        if len(list_key_tuple_answers) == 0: return None

        # Extract probability tensor
        tensor_probs = torch.tensor([rel for (_, _, _, rel) in list_key_tuple_answers], dtype=torch.float32)

        # Sample through the tensor
        dict_multinomial = {
            "input": tensor_probs,
            "num_samples": 1,
            "replacement": True
        }
        if generator:
            dict_multinomial["generator"] = generator
        i = torch.multinomial(**dict_multinomial).item()

        # Get the key using the sampled index
        key = list_key_tuple_answers[i][0]

        # Split the key at the separator ("+")
        list_keys = key.split(separator)

        # Iterate list_keys as long as there is no end token (</s>)
        for _key in list_keys[len(list_keys) - 1:]:
            if _key == END_TOKEN:
                max_index += 1
                print("")
            else:
                print(_key, end=" ")

        # arg is the key replaced by the former arg and the separator
        arg = "+".join(list_keys[1:])
        if (max_index >= max_lines):
            print("\nEND")
            break

predict(list_list_words, "the+club+isnt")

# Possible questions:
# - Can you generate the same output with both wihtout and with the seed?
# - How to generate accurate song lyrics from Shape of You by Ed Sheeran?
# - Why does your generation stop abruptly?

START

the club isnt the best place to find a lover 
so the heart falls where your chest is 
now a piece of me is you 
lately everythings making sense too 
baby im dancing in the dark with you between my arms 
barefoot on the grass while listenin to our favorite song 
when you could have anyone else 
oh you are my one and only 
and you dont care bout where ive been 
you know it aint my time to call it a day 

END
